# Building a CNN architecture using Tensorflow

* Dataset - We are using [MNIST](http://yann.lecun.com/exdb/mnist/) dataset for our model. 
* It is a digit recognition task in which we identify digits (handwritten) from 0 to 9.
* Each image is a 28x28 pixel square (784 pixels in total)

# CNN architecture 

* Convolution, Filter shape (5x5x6), Stride = 1, Padding = same
* Max Pooling (2x2), Window shape (2x2), Stride = 2, Padding = same
* ReLU

* Convolution, Filter shape (5x5x16), Stride=1, Padding = same
* Max Pooling (2x2), Window shape (2x2), Stride = 2, Padding = same
* ReLU

* Fully connected layers (128)
* ReLU
* Fully connected layers (10)
* Softmax

## 1. Importing Libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
import time

## 2. Load data

In [4]:
from tensorflow.examples.tutorials.mnist import input_data

data = input_data.read_data_sets('data/MNIST/', one_hot=True)

Extracting data/MNIST/train-images-idx3-ubyte.gz
Extracting data/MNIST/train-labels-idx1-ubyte.gz
Extracting data/MNIST/t10k-images-idx3-ubyte.gz
Extracting data/MNIST/t10k-labels-idx1-ubyte.gz


In [6]:
print('Size of: ')
print('- Training Set:\t\t{}'.format(len(data.train.labels)))
print('- Test Set:\t\t{}'.format(len(data.test.labels)))
print('- Validation Set:\t{}'.format(len(data.validation.labels)))

Size of: 
- Training Set:		55000
- Test Set:		10000
- Validation Set:	5000


## 3. Placeholder variables

In [7]:
#Placeholder variable for the input imaage
x = tf.placeholder(tf.float32, shape=[None, 28*28], name = 'X')

#Reshape it into [num_images, image_height, image_width, num_channels]

x_image = tf.reshape(x, [-1, 28, 28, 1])

#Placeholder variables for the true labels associated with the images
y_true = tf.placeholder(tf.float32, shape =[None, 10], name = 'y_true')

y_true_cls = tf.argmax(y_true, dimension=1)

Instructions for updating:
Use the `axis` argument instead


## 4. Function for creating a new Convolutional Layer

In [8]:
def new_conv_layer(input, num_input_channels, filter_size, num_filters, name):
    with tf.variable_scope(name) as scope:
        #Shape of the filter weights for the convolution
        shape = [filter_size, filter_size, num_input_channels, num_filters]
        
        #Create new weights (filters) with the given shape
        weights = tf.Variable(tf.truncated_normal(shape, stddev=0.05))
        
        #Create new biases, one for each filter
        biases = tf.Variable(tf.constant(0.05, shape=[num_filters]))
        
        #Tensorflow operation for Convolution
        layer = tf.nn.conv2d(input = input, filter = weights, strides = [1,1,1,1], padding = 'SAME')
        
        #Add the biases to the result of the Convolution
        layer += biases
        
        return layer, weights

## 5. Function for creating a new pooling layer

In [9]:
def new_pool_layer(input, name):
    with tf.variable_scope(name) as scope:
        
        #tensorflow operation for convolution
        layer = tf.nn.max_pool(value = input, ksize = [1, 2, 2, 1], strides = [1,2,2,1], padding='SAME')
        
        return layer

## 6. Function for creating a new ReLU Layer

In [10]:
def new_relu_layer(input, name):
    with tf.variable_scope(name) as scope:
        
        #tensorflow operation for convolution
        layer = tf.nn.relu(input)
        
        return layer

## 7. Function to create a fully connected layer

In [11]:
def new_fc_layer(input, num_inputs, num_outputs, name):
    
    with tf.variable_scope(name) as scope:
        
        #Create new weights and biases
        weights = tf.Variable(tf.truncated_normal([num_inputs, num_outputs], stddev = 0.05))
        
        biases = tf.Variable(tf.constant(0.05, shape = [num_outputs]))
        
        #Multiply the input layers and the weights and then add the bias values
        layer = tf.matmul(input, weights) + biases
        
        return layer

## 8. Create the Convolutional Neural Network

In [13]:
#Convolution layer 1
layer_conv1, weights_conv1 = new_conv_layer(input = x_image, num_input_channels = 1, filter_size = 5, num_filters = 6, name = 'Conv1')

#Pooling layer 1
layer_pool1 = new_pool_layer(layer_conv1, name = 'pool1')

#ReLU layer 1
layer_relu1 = new_relu_layer(layer_pool1, name = 'relu1')

#Convolution layer 2
layer_conv2, weights_conv2 = new_conv_layer(input = layer_relu1, num_input_channels=6, filter_size = 5, num_filters=16, name = 'Conv2')

#Pooling layer 2
layer_pool2 = new_pool_layer(layer_conv2, name = 'pool2')

#ReLU layer 2
layer_relu2 = new_relu_layer(layer_pool2, name = 'relu2')

#Flatten layer
num_features = layer_relu2.get_shape()[1:4].num_elements()
layer_flat = tf.reshape(layer_relu2, [-1, num_features])

#Fully-connected layer 1
layer_fc1 = new_fc_layer(layer_flat, num_inputs=num_features, num_outputs=128, name = 'fc1')

#ReLU layer 3
layer_relu3 = new_relu_layer(layer_fc1, name = 'relu3')

#Fully-connected layer 2
layer_fc2 = new_fc_layer(input = layer_relu3, num_inputs=128, num_outputs=10, name='fc2')


In [14]:
#Use softmax function to normalize the output

with tf.variable_scope('Softmax'):
    y_pred = tf.nn.softmax(layer_fc2)
    y_pred_cls = tf.argmax(y_pred, dimension = 1)

## 9. Cost Function

In [15]:
#Use cross entropy cost function
with tf.name_scope('cross_ent'):
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits = layer_fc2, labels = y_true)
    
    cost = tf.reduce_mean(cross_entropy)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



## 10. Optimizer

In [16]:
#Use adam optimizer
with tf.name_scope('optimizer'):
    optimizer = tf.train.AdamOptimizer(learning_rate = 1e-4).minimize(cost)

## 11. Accuracy

In [17]:
with tf.name_scope('accuracy'):
    correct_prediction = tf.equal(y_pred_cls, y_true_cls)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [18]:
# Initialize the file writer
writer = tf.summary.FileWriter('Training FileWriter/')
writer1 = tf.summary.FileWriter('Validation_FileWriter/')

In [23]:
#Add the cost and accuracy to the summary
tf.summary.scalar('loss', cost)
tf.summary.scalar('accuracy', accuracy)

#Merge all summaries together
merged_summary = tf.summary.merge_all()

In [20]:
num_epochs = 100
batch_size = 100

## 12. TensorFlow Session

In [24]:
with tf.Session() as sess:
    #Initialize all variables
    sess.run(tf.global_variables_initializer())
    
    #Add the model graph to TensorBoard
    writer.add_graph(sess.graph)
    
    #Loop over number of epochs
    for epoch in range(num_epochs):
        start_time = time.time()
        train_accuracy = 0
        
        for batch in range(0, int(len(data.train.labels)/batch_size)):
            #Get a batch of images and labels
            x_batch, y_true_batch = data.train.next_batch(batch_size)
            
            #Put a batch into a dictionary with proper names for placeholder variables
            feed_dict_train = {x: x_batch, y_true: y_true_batch}
            
            #Run the optimizer using this batch of training set
            sess.run(optimizer, feed_dict = feed_dict_train)
            
            #Calculate the accuracy of the batch of the training data
            train_accuracy += sess.run(accuracy, feed_dict = feed_dict_train)
            
            #Generate summary with the current batch of data and write to the file
            summ = sess.run(mergerd_summary, feed_dict = feed_dict_train)
            
            writer.add_summary(summ, epoch*int(len(data.train.labels)/ batch_size) + batch)
            
        train_accuracy /= int(len(data.train.labels)/batch_size)
            
        #Generate summary and validate the model on entire validation set
        summ, vali_accuracy = sess.run([merged_summary, accuracy], feed_dict = {x: data.validation.images, y_true: data.validation.labels})
        writer1.add_summary(summ, epoch)
            
        end_time = time.time()
        
        print('Epoch '+str(epoch+1)+ 'completed: Time Usage: '+str(int(end_time-start_time))+' seconds')
        print('\tAccuracy:')
        print('\t- Training Accuracy:\t{}'.format(train_accuracy))
        print('\t- Validation Accuracy:\t{}'.format(vali_accuracy))

Epoch 1completed: Time Usage: 67 seconds
	Accuracy:
	- Training Accuracy:	0.7209636353362691
	- Validation Accuracy:	0.881600022315979
Epoch 2completed: Time Usage: 62 seconds
	Accuracy:
	- Training Accuracy:	0.8962363645163449
	- Validation Accuracy:	0.9154000282287598
Epoch 3completed: Time Usage: 62 seconds
	Accuracy:
	- Training Accuracy:	0.9187272746996447
	- Validation Accuracy:	0.9333999752998352
Epoch 4completed: Time Usage: 60 seconds
	Accuracy:
	- Training Accuracy:	0.9326000023971904
	- Validation Accuracy:	0.9449999928474426
Epoch 5completed: Time Usage: 65 seconds
	Accuracy:
	- Training Accuracy:	0.9438909107988531
	- Validation Accuracy:	0.9539999961853027
Epoch 6completed: Time Usage: 74 seconds
	Accuracy:
	- Training Accuracy:	0.9527090926603837
	- Validation Accuracy:	0.9592000246047974
Epoch 7completed: Time Usage: 66 seconds
	Accuracy:
	- Training Accuracy:	0.9605272775346583
	- Validation Accuracy:	0.965399980545044
Epoch 8completed: Time Usage: 69 seconds
	Accuracy